In [ ]:
import pandas as pd
import numpy as np
import pickle

In [ ]:

csitrade=pd.read_csv('stock/NASDAQ100_new.csv')
if 'Close' in csitrade.columns:
    csitrade.rename(columns={'Close':'close'},inplace=True)
# The data must be sorted by stock.
csitrade['close_r']=csitrade['close']/csitrade['close'].shift(1)

# csitrade=pd.read_csv('../HS300_10_24/HS300A.csv')
# csitrade.rename(columns={'label':'close_r'},inplace=True)

In [ ]:
csitrade.reset_index(inplace=True,drop=True)

In [ ]:
#market and daily return
import pandas as pd
df_trade=csitrade
df_trade.rename(columns={'kdcode':'instrument','dt':'datetime'},inplace=True)
df_trade.rename(columns={'company':'instrument','Date':'datetime'},inplace=True)
df_trade['datetime'] = df_trade['datetime'].astype('datetime64[ns]')

In [ ]:
df_trade=df_trade[df_trade['datetime']>'2023-01-01']
# df_trade=df_trade[df_trade['datetime']<='2024-06-30']
df_trade.reset_index(inplace=True,drop=True)

In [ ]:
index_date_unique=df_trade['datetime'].unique()

In [ ]:
df=pd.read_csv('pred.csv')
df.rename(columns={'label':'score'},inplace=True)
# df.rename(columns={'dt':'datetime','kdcode':'instrument'},inplace=True)
df['datetime']=df['datetime'].astype('datetime64[ns]')

df_score=pd.DataFrame()
for date in index_date_unique[:-1]:
    loc=np.where(index_date_unique==date)
    nextdate_loc=np.where(index_date_unique==date)[0][0]+1
    for i in range(0,len(df)):
        if df['datetime'][i]== date:
            b={'datetime':index_date_unique[nextdate_loc],'instrument':df['instrument'][i],'score':df['score'][i]}
            df_score=df_score._append(b,ignore_index=True)

In [ ]:
# df_score = df[df['datetime'].isin(index_date_unique)].copy()
x=pd.merge(df_trade,df_score,on=['datetime','instrument'],how='outer')
trade_date_unique=df_trade['datetime'].unique()
df_return=pd.DataFrame()
b={'datetime':trade_date_unique[0],'daily_return':0}
df_return=df_return._append(b,ignore_index=True)
for date in trade_date_unique[1:]:
    x_day=x[x['datetime']==date]
    r_day=x_day.nlargest(9,columns='score').close_r.mean()
    r_day=r_day-1
    #r_day-=x_day.nsmallest(3,columns='score').close_r.mean()
    b={'datetime':date,'daily_return':r_day}
    df_return=df_return._append(b,ignore_index=True)

In [ ]:
portfolio_df_performance = df_return.set_index(['datetime'])
portfolio_df_performance

In [ ]:
ARR = (1 + portfolio_df_performance).cumprod()
ARR

In [ ]:
t = df_trade['close_r'].values.reshape(-1, df_trade['datetime'].nunique())
t  = np.mean(t, axis=0)
t = t.cumprod()

In [ ]:
data = []
data.append(ARR['daily_return'].values)
data.append(t)
data = np.array(data)
data.shape

In [ ]:
'''
import matplotlib.pyplot as plt

def draw(data, meaning, title, color=['red', 'blue', 'green', 'orange', 'purple'], ncol=1):
    plt.figure(figsize=(6, 2), dpi=200)
    for i in range(data.shape[0]):
        plt.plot(np.linspace(0.025, 1.965, data.shape[1]), data[i], label=f"{meaning[i]}", color=color[i])

    plt.fill_between(np.linspace(0.025, 1.965, data.shape[1]), data[0], data[1], color='#ffffcc', alpha=0.5, label='Excess Return')

    plt.xlabel('Date')
    plt.ylabel('Accumulative Return')

    date_labels = ['2024-01-01', '2024-03-01', '2024-05-01', '2024-06-30']
    plt.xticks(np.linspace(0.0, 2.0, len(date_labels)), date_labels)
    for x in np.linspace(0.0, 2.0, len(date_labels)):
        plt.axvline(x=x, color='lightgray', linestyle='--', linewidth=1)

    plt.legend(title=title, ncol=ncol, loc='best', fontsize='small')
    plt.show()

meaning = ['StockMamba', 'Market']
title = ''
draw(data, meaning, title)
'''

In [ ]:
alpha_df_performance = pd.DataFrame()
alpha_df_performance['portfolio_daily_return'] = portfolio_df_performance['daily_return']
alpha_df_performance['portfolio_net_value'] = (alpha_df_performance['portfolio_daily_return'] + 1).cumprod()

net_value_columns = ['portfolio_net_value']

alpha_statistics_df = pd.DataFrame(index=alpha_df_performance[net_value_columns].columns,
                                    columns=["ARR", "AVoL", "MDD", "ASR", "Calmar", "IR"])

# alpha_df_performance.set_index("dt", inplace=True)
alpha_df_performance.index = pd.to_datetime(alpha_df_performance.index)
monthly_statistics_df = alpha_df_performance[net_value_columns].resample('m').last()
monthly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    monthly_statistics_df])
monthly_statistics_df = monthly_statistics_df.pct_change()
monthly_statistics_df = monthly_statistics_df.dropna()
monthly_statistics_df.index = monthly_statistics_df.index.date

yearly_statistics_df = alpha_df_performance[net_value_columns].resample('y').last()
yearly_statistics_df = pd.concat([alpha_df_performance[:1][
                                        ['portfolio_net_value']],
                                    yearly_statistics_df])
yearly_statistics_df = yearly_statistics_df.pct_change()
yearly_statistics_df = yearly_statistics_df.dropna()
yearly_statistics_df.index = yearly_statistics_df.index.date

alpha_statistics_df.loc[:, "ARR"] = np.mean(
    (alpha_df_performance[net_value_columns].tail(1)) ** (252 / len(alpha_df_performance)) - 1)
alpha_statistics_df.loc[:, "AVoL"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252)
alpha_statistics_df.loc[:, "Cumulative Return"] = np.mean(alpha_df_performance[net_value_columns].tail(1) - 1)
alpha_statistics_df.loc[:, "Cumulative Volatility"] = np.std(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)
alpha_statistics_df.loc[:, "MDD"] = np.min(
    (alpha_df_performance[net_value_columns] - alpha_df_performance[net_value_columns].cummax()) /
    alpha_df_performance[net_value_columns].cummax())
alpha_statistics_df.loc[:, "ASR"] = alpha_statistics_df["ARR"] / alpha_statistics_df["AVoL"]
alpha_statistics_df.loc[:, "Calmar"] = alpha_statistics_df["ARR"] / abs(alpha_statistics_df["MDD"])
alpha_statistics_df.loc[:, "IR"] = np.mean(
    alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1) * np.sqrt(
    252) / np.std(alpha_df_performance[net_value_columns] / alpha_df_performance[net_value_columns].shift(1) - 1)

In [ ]:
alpha_statistics_df

In [ ]:
# ARR.to_csv(f"para_sensitivity/nasdaq_mamba=2.csv")